# 04 — TraCI Bridge Sanity Check (Simulation Manager)

## Objective
Verify we can:
1) load SUMO runtime config
2) start a SUMO process via TraCI
3) step the simulation
4) close cleanly

This notebook is a minimal connectivity test before we build routing, vehicles, and control agents.

## 1.0 Notebook Rules

- All imports are in the first code cell only.
- Logic lives in `src/` (reusable modules).
- The notebook focuses on orchestration + readable prints.

In [4]:
import sys
import json
from pathlib import Path

import traci  # ensures TraCI import is available in notebook

# Ensure repo root import works (so `src/...` imports work in notebooks)
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.sim.simulation_manager import SimulationManager, SumoConfig

SUMO_RUNTIME_CFG = PROJECT_ROOT / "configs" / "sumo_config.json"

print("PROJECT_ROOT:", PROJECT_ROOT)
print("SUMO_RUNTIME_CFG:", SUMO_RUNTIME_CFG)
print("TraCI imported OK")

PROJECT_ROOT: /Users/testsolutions/Documents/Academics/mission-capstone/marl-in-ems
SUMO_RUNTIME_CFG: /Users/testsolutions/Documents/Academics/mission-capstone/marl-in-ems/configs/sumo_config.json
TraCI imported OK


## 2.0 Load SUMO runtime config and build SimulationManager

We will:
- load `configs/sumo_config.json`
- create a `SumoConfig`
- point to a minimal `.sumocfg` (`sim/config/kigali_test.sumocfg`)

In [5]:
SUMOCFG_PATH = PROJECT_ROOT / "sim" / "config" / "kigali_test.sumocfg"
print("SUMOCFG_PATH:", SUMOCFG_PATH)
print("Exists:", SUMOCFG_PATH.exists())

with open(SUMO_RUNTIME_CFG, "r", encoding="utf-8") as f:
    runtime_cfg = json.load(f)

cfg = SumoConfig(
    use_gui=bool(runtime_cfg.get("use_gui", False)),
    step_length=float(runtime_cfg.get("step_length", 1.0)),
    seed=int(runtime_cfg.get("seed", 42)),
    sumo_bin=runtime_cfg.get("sumo_bin"),
    sumo_gui_bin=runtime_cfg.get("sumo_gui_bin"),
)

print("\n## Runtime config")
print(cfg)

sim = SimulationManager(config=cfg, sumocfg_path=SUMOCFG_PATH)
print("\n[OK] SimulationManager ready")

SUMOCFG_PATH: /Users/testsolutions/Documents/Academics/mission-capstone/marl-in-ems/sim/config/kigali_test.sumocfg
Exists: True

## Runtime config
SumoConfig(use_gui=False, step_length=1.0, seed=42, sumo_bin=None, sumo_gui_bin=None)

[OK] SimulationManager ready


## 3.0 Start SUMO, step, and close

This is a minimal lifecycle test:
- start TraCI
- step 100 ticks
- print simulation time
- close cleanly

In [6]:
try:
    sim.start()

    print("\n## Stepping 100 ticks...")
    sim.step(100)

    t = sim.get_sim_time()
    print("[OK] sim time (s):", t)

finally:
    sim.close()

## Starting SUMO
Binary: /Library/Frameworks/EclipseSUMO.framework/Versions/1.26.0/EclipseSUMO/bin/sumo
Config : /Users/testsolutions/Documents/Academics/mission-capstone/marl-in-ems/sim/config/kigali_test.sumocfg
Cmd    : /Library/Frameworks/EclipseSUMO.framework/Versions/1.26.0/EclipseSUMO/bin/sumo -c /Users/testsolutions/Documents/Academics/mission-capstone/marl-in-ems/sim/config/kigali_test.sumocfg --step-length 1.0 --seed 42 --no-warnings true
 Retrying in 1 seconds


[OK] TraCI connected.

## Stepping 100 ticks...
[OK] sim time (s): 100.0
## Closing TraCI
Step #100.00 (0ms ?*RT. ?UPS, TraCI: 1ms, vehicles TOT 0 ACT 0 BUF 0)                      
[OK] TraCI closed.
